Import Libraries and Set Up:

In [1]:
!pip install langchain langchain_community==0.2.6 pypdf fastembed==0.3.2 chromadb

  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached mmh3-4.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.6 MB/s eta 0:00:00
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using

In [3]:
#get the huggingface token
import os
from getpass import getpass

HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

Huggingface Token : ··········


Load and Split the PDF Document:

In [4]:
!wget https://ncert.nic.in/textbook/pdf/lekl101.pdf

--2024-07-09 19:51:14--  https://ncert.nic.in/textbook/pdf/lekl101.pdf
Resolving ncert.nic.in (ncert.nic.in)... 164.100.166.133
Connecting to ncert.nic.in (ncert.nic.in)|164.100.166.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 866980 (847K) [application/pdf]
Saving to: ‘lekl101.pdf.2’

lekl101.pdf.2       100%[===================>] 846.66K   312KB/s    in 2.7s    

2024-07-09 19:51:18 (312 KB/s) - ‘lekl101.pdf.2’ saved [866980/866980]



In [5]:
# import PDF reader
from langchain_community.document_loaders.pdf import PyPDFLoader

# load the document
# Source credits: https://ncert.nic.in/textbook/pdf/lekl101.pdf
loader = PyPDFLoader("lekl101.pdf")
data = loader.load()

In [6]:
data

[Document(metadata={'source': 'lekl101.pdf', 'page': 0}, page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short\nstory’ of about five hundred words to longer and\nmore complex works. The longer ones, with their\nstatus of middle length, fall between the tautness\nof the short narrative and the expansiveness of\nthe novel.\nThere can be thematic variation too. The stories\ndeal with fantasy, reality, alienation and the\nproblem of choice in personal life. There are three\nshort stories and two long ones in this sectio

In [7]:
# import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 0,
)

chunks = text_splitter.split_documents(data)

In [8]:
chunks[0]

Document(metadata={'source': 'lekl101.pdf', 'page': 0}, page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short')

 Generate Text Embeddings:

In [9]:
import langchain_community
import fastembed

/Users/jloscalzo/Projects/pruebas/agents-llms-planet-ai/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
langchain_community.__version__, fastembed.__version__

('0.2.6', '0.3.2')

In [13]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
# supported models: https://qdrant.github.io/fastembed/examples/Supported_Models/
model_name = "thenlper/gte-large"
model_name="BAAI/bge-small-en-v1.5"
embedding_model = FastEmbedEmbeddings(model_name=model_name)


Fetching 5 files: 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]


Create and Initialize Vector Store:

In [14]:
embedding_model.embed_query("what is the importance of I sell my dreams according to the author")

[0.03516080230474472,
 -0.0004281291621737182,
 0.018682900816202164,
 -0.001173773198388517,
 0.012892977334558964,
 0.023913579061627388,
 0.059293895959854126,
 0.05672737583518028,
 0.02829848788678646,
 -0.005674541927874088,
 -0.013736896216869354,
 0.012205484323203564,
 -0.013972264714539051,
 -0.02510213479399681,
 -0.005438315682113171,
 0.022188691422343254,
 0.03372221812605858,
 -0.022699421271681786,
 -0.09241140633821487,
 -0.01685389317572117,
 0.03303595632314682,
 -0.004605232737958431,
 0.014025077223777771,
 0.01028556190431118,
 0.056247856467962265,
 -0.029521489515900612,
 -0.03355337306857109,
 0.05392352119088173,
 -0.05623311176896095,
 -0.13104315102100372,
 -0.029519617557525635,
 -0.010687019675970078,
 0.023154517635703087,
 -0.037008579820394516,
 0.039224714040756226,
 0.051940642297267914,
 -0.056903135031461716,
 0.021277908235788345,
 0.01374431699514389,
 0.00921107642352581,
 0.1061229556798935,
 0.0010553904576227069,
 -0.006966502871364355,
 0.015

In [15]:
from langchain.vectorstores import Chroma

In [16]:
# initialize the vector store (save to disk)
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

 Define User Query:

In [17]:
# Let's define the query, since we are going to use it multiple times.
query = "what is importance of I sell my dreams according to the author"

Retrieve Documents and Initialize Retriever:

In [18]:
# retrieve from vector db (load from disk) with query
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)
retrieved_docs = db2.similarity_search(query)
print(retrieved_docs[0].page_content)

10/KALEIDOSCOPE
Language W ork
A.Vocabulary
Look up the meanings of the following phrases under ‘dream’
and ‘sell’ in the dictionary
dream         sell
dream on sell-by date
dream something away selling-point
(not) dream of doing something sell-out
dream something up selling price
look like a dream seller’s market
B.Grammar: Emphasis
Read this sentence carefully
One morning  at nine o’clock, while we were having
breakfast on the terrace of the Havana Riviera Hotel


In [19]:
retrieved_docs[0]


Document(metadata={'page': 9, 'source': 'lekl101.pdf'}, page_content='10/KALEIDOSCOPE\nLanguage W ork\nA.Vocabulary\nLook up the meanings of the following phrases under ‘dream’\nand ‘sell’ in the dictionary\ndream         sell\ndream on sell-by date\ndream something away selling-point\n(not) dream of doing something sell-out\ndream something up selling price\nlook like a dream seller’s market\nB.Grammar: Emphasis\nRead this sentence carefully\nOne morning  at nine o’clock, while we were having\nbreakfast on the terrace of the Havana Riviera Hotel')

In [20]:
# initialize the retriever
# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/mmr/
retriever = db2.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

Define LLM and Chain Components:

In [80]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   !curl -fsSL https://ollama.com/install.sh | sh
   !ollama serve > rocama.log 2>&1 &
   !ollama pull mistral:7b
else:
   print("NOT in Colab")

Running in Colab
>>> Downloading ollama...
###########################################                                                    47.2%^C
Error: could not connect to ollama app, is it running?


In [21]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral:7b")

In [22]:
llm.invoke("what is the importance of I sell my dreams according to the author")

' In the poem "I Will Not Sing" by Langston Hughes, the phrase "I sell my dreams" appears as a metaphor for the struggle and frustration experienced by African Americans in trying to achieve their goals and aspirations in a society that is discriminatory and oppressive.\n\nThe importance of "not selling one\'s dreams," according to Hughes, lies in the fact that dreams represent hope, ambition, and the desire for a better future. Selling one\'s dreams means giving up on these aspirations and settling for a life of mediocrity or even suffering. It is a sad and disheartening reality for many people who face systemic barriers to success.\n\nHowever, Hughes also recognizes that holding onto dreams alone is not enough. Dreams must be accompanied by action and determination. In the poem, he writes "But I keep on trying / I will not give in" despite the challenges and setbacks he faces. The importance of selling one\'s dreams lies in the fact that they serve as a source of motivation and inspi

In [69]:
# import the question-answering chain and Huggingface Hub LLM
# from langchain.llms import HuggingFaceHub
# model_path = "mistralai/Mistral-7B-Instruct-v0.2"

# # # define the llm
# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.2",
#     model_kwargs={
#       "temperature":0.1,
#       "max_new_tokens":512,
#       "return_full_text":False,
#       "repetition_penalty":1.1,
#       "top_p":0.9,
#     },
# )

# from langchain.llms import HuggingFacePipeline
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForCausalLM, pipeline

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

# pipe = pipeline(
#     "text-generation",
#     model=model_path,
#     tokenizer=model_path,
#     max_new_tokens=512,
#     temperature=0.1,
#     top_p=0.9,
#     repetition_penalty=1.1,
#     truncation=True,
#     return_full_text=False,
# )

# local_llm = HuggingFacePipeline(pipeline=pipe)



In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [24]:
template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""

In [25]:
prompt = ChatPromptTemplate.from_template(template)

In [30]:
prompt

ChatPromptTemplate(input_variables=['context', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template="\n<s>[INST]\nYou are an AI Assistant that follows instructions extremely well.\nPlease be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT\n[/INST]\nCONTEXT: {context}\n</s>\n[INST]\n{query}\n[/INST]\n"))])

In [26]:
output_parser = StrOutputParser()

In [27]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

Run the Chain and Get Response:

In [28]:
query

'what is importance of I sell my dreams according to the author'

In [29]:
response = chain.invoke(query)
response

' Based on the context provided, "I sell my Dreams" is not explicitly discussed by the author in terms of its importance. The phrase appears multiple times in a repetitive manner in the text, but there\'s no interpretation or commentary provided by the author regarding its meaning or significance.'

In [31]:
response2 = chain.invoke("what is the author name?")
response2

" The author's name is Gabriel Garcia Marquez. This information can be found in the context provided in the document with the metadata 'source': 'lekl101.pdf', page_content: 'Gabriel Garcia Marquez was brought up by his grandparents in Northern Columbia because his parents were poor and struggling. A novelist, short-story writer and journalist, he is widely considered the greatest living Latin American master of narrative.'"

In [32]:
response3 = chain.invoke("for how many years does the Author refer the lady to leave right away and not come back to Vienna")
response3

' The author is referred to leave Vienna for five years based on what the lady said in the text.'

In [ ]:
'The text mentions that thirteen years had passed since the first meeting between the lady and the narrator (page 6, document 1). However, there is no explicit statement telling the lady to leave Vienna right away and not come back during this conversation. The lady expresses her intention not to return to Vienna even if her dreams turn out to be false (page 6, document 1). But there is no indication that anyone asks her to leave immediately or that she leaves at that moment. Therefore, based on the provided context, I cannot directly answer your question with certainty.'
